<a href="https://colab.research.google.com/github/hybrid18581/Pinball/blob/master/skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!ls "/content/drive/MyDrive/"

 20231021_090717315_iOS.heic
 20231021_090807040_iOS.heic
 20231021_101234649_iOS.heic
 20231021_101238438_iOS.heic
 20231021_101929279_iOS.heic
 688363e824201cc92596e7bc2d31891c2dd8c878c4c6bf65109528c861f869a4.pdf
 A96F388ADF040024B6A402B00_copy.pdf
'Acct Statement_XX2041_07052025.pdf'
'Colab Notebooks'
 EAadhaar_2836072801275420191213122602_14112022215234-1.pdf
 id_docs.pdf
'images (8).jpeg'
'IMG_20250822_093546_344_AI (1).jpg'
 IMG_20250822_093546_344_AI.jpg
 IMG-20250822-WA0004.jpg
 IMG_2494.HEIC
 IMG_2495.HEIC
 IMG_2500.HEIC
 IMG_2501.HEIC
 IMG_2502.HEIC
 IMG_2503.HEIC
 IMG_2504.HEIC
 IMG_2505.HEIC
 IMG_2506.HEIC
 IMG_2507.HEIC
 Leaderboard.gsheet
'Meteora POC (version 2) (1).xlsx'
'Meteora POC (version 2).xlsx'
'Meteora Pricing.gsheet'
 models
 Netflix_datavisualisation_Sidhanth.pdf
'New Microsoft Word Document.docx'
 payment_confirmation_202508234-095358.png
'Resume_ds (1).pdf'
'Resume_ds (2).pdf'
'Resume_ds (3).pdf'
'Resume_ds (4).pdf'
 resume_ds.pdf
 Resume_ds.pdf
 Resume_Sidh

In [21]:
!cp -r "/content/drive/MyDrive/Skin_Disease_Dataset.zip" /content/

In [22]:
!unzip -q "/content/Skin_Disease_Dataset.zip" -d /content/skin_dataset

In [23]:
!ls "/content/skin_dataset/skin Disease Dataset/kaggle"

test  train  val


In [24]:
train_path = "/content/skin_dataset/skin Disease Dataset/kaggle/train"
val_path   = "/content/skin_dataset/skin Disease Dataset/kaggle/val"
test_path  = "/content/skin_dataset/skin Disease Dataset/kaggle/test"

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.amp import autocast, GradScaler

In [26]:
!mkdir -p /content/drive/MyDrive/models

In [27]:
import os
import shutil
import numpy as np
from torchvision import datasets

NUM_CLASSES = 6
SAMPLES_PER_CLASS = 3000

train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train")
targets = np.array(train_dataset.targets)

balanced_dir = "/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced"
os.makedirs(balanced_dir, exist_ok=True)

if os.path.exists(balanced_dir):
    shutil.rmtree(balanced_dir)

os.makedirs(balanced_dir)

for cls in range(NUM_CLASSES):
    cls_indices = np.where(targets == cls)[0]
    selected = np.random.choice(cls_indices, SAMPLES_PER_CLASS, replace=len(cls_indices)<SAMPLES_PER_CLASS)

    class_dir = os.path.join(balanced_dir, train_dataset.classes[cls])
    os.makedirs(class_dir, exist_ok=True)

    for j, idx in enumerate(selected):
     src_path = train_dataset.imgs[idx][0]

     filename = f"{j}_{os.path.basename(src_path)}"
     dst_path = os.path.join(class_dir, filename)

     shutil.copy(src_path, dst_path)


In [28]:
!ls "/content/skin_dataset/skin Disease Dataset/kaggle"

test  train  train_balanced  val


In [29]:

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [30]:
train_dataset = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/train_balanced", transform=train_transforms)
val_dataset   = datasets.ImageFolder("/content/skin_dataset/skin Disease Dataset/kaggle/val", transform=val_transforms)

BATCH_SIZE = 64

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          shuffle=True, num_workers=2)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)

NUM_CLASSES = 6

In [31]:
import torch
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
model_rn = resnet50(weights=weights)
model_rn.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model_rn.fc.in_features, 1024),
    nn.BatchNorm1d(1024),
    nn.ReLU(inplace=True),

    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(inplace=True),

    nn.Dropout(0.3),
    nn.Linear(512, NUM_CLASSES)
)

for param in model_rn.parameters():
    param.requires_grad = False

for param in model_rn.fc.parameters():
    param.requires_grad = True

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_rn = model_rn.to(DEVICE)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 117MB/s]


In [32]:
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(
    model_rn.fc.parameters(),
    lr=1e-5,
    weight_decay=1e-4
)


In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score
EPOCHS = 25
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
counter = 0
best_val_f1 = 0
MODEL_PATH = "/content/drive/MyDrive/models/resnet50_1.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_rn.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_rn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_rn.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_rn(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_rn.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

Epoch 1/25 | Batch 282/282 | Loss: 1.5666 | Acc: 0.5000 | Prec: 0.4417 | Rec: 0.4417 | F1: 0.3988

Epoch 1 Summary
Train | Loss: 1.6446 | Acc: 0.3258 | Prec: 0.3429 | Rec: 0.3258 | F1: 0.3172
Val   | Acc: 0.5325 | Prec: 0.4850 | Rec: 0.5413 | F1: 0.4836
Model improved. Saved.
Epoch 2/25 | Batch 282/282 | Loss: 1.2438 | Acc: 0.5000 | Prec: 0.5000 | Rec: 0.4444 | F1: 0.4365

Epoch 2 Summary
Train | Loss: 1.3621 | Acc: 0.4861 | Prec: 0.4858 | Rec: 0.4861 | F1: 0.4841
Val   | Acc: 0.5539 | Prec: 0.5077 | Rec: 0.5759 | F1: 0.5122
Model improved. Saved.
Epoch 3/25 | Batch 282/282 | Loss: 1.0475 | Acc: 0.6875 | Prec: 0.6806 | Rec: 0.7222 | F1: 0.6873

Epoch 3 Summary
Train | Loss: 1.2326 | Acc: 0.5292 | Prec: 0.5287 | Rec: 0.5292 | F1: 0.5279
Val   | Acc: 0.5799 | Prec: 0.5268 | Rec: 0.5916 | F1: 0.5315
Model improved. Saved.
Epoch 4/25 | Batch 282/282 | Loss: 0.9919 | Acc: 0.5625 | Prec: 0.4583 | Rec: 0.6667 | F1: 0.5214

Epoch 4 Summary
Train | Loss: 1.1525 | Acc: 0.5591 | Prec: 0.5585 | Re

In [34]:
for param in model_rn.layer4.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam([
    {"params": model_rn.fc.parameters(), "lr": 1e-5},
    {"params": model_rn.layer4.parameters(), "lr": 1e-6}
], weight_decay=1e-4)


In [35]:
from sklearn.metrics import precision_score, recall_score, f1_score
EPOCHS = 15
BATCH_SIZE = 64
PATIENCE = 5
best_val_acc = 0
counter = 0
best_val_f1 = 0
MODEL_PATH = "/content/drive/MyDrive/models/resnet50_2.pth"

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

for epoch in range(EPOCHS):

    model_rn.train()

    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for i, (inputs, labels) in enumerate(train_loader, 1):

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model_rn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, 1)

        preds_np = preds.detach().cpu().numpy()
        labels_np = labels.detach().cpu().numpy()

        all_train_preds.extend(preds_np)
        all_train_labels.extend(labels_np)

        running_loss += loss.item() * inputs.size(0)

        batch_acc = (preds_np == labels_np).mean()
        batch_precision = precision_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_recall = recall_score(labels_np, preds_np, average='macro', zero_division=0)
        batch_f1 = f1_score(labels_np, preds_np, average='macro', zero_division=0)

        sys.stdout.write(
            f"\rEpoch {epoch+1}/{EPOCHS} | "
            f"Batch {i}/{len(train_loader)} | "
            f"Loss: {loss.item():.4f} | "
            f"Acc: {batch_acc:.4f} | "
            f"Prec: {batch_precision:.4f} | "
            f"Rec: {batch_recall:.4f} | "
            f"F1: {batch_f1:.4f}"
        )
        sys.stdout.flush()

    print()

    train_loss = running_loss / len(train_dataset)
    train_acc = np.mean(np.array(all_train_preds) == np.array(all_train_labels))
    train_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
    train_f1 = f1_score(all_train_labels, all_train_preds, average='macro', zero_division=0)

    model_rn.eval()

    val_preds = []
    val_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model_rn(inputs)
            preds = torch.argmax(outputs, 1)

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_acc = np.mean(np.array(val_preds) == np.array(val_labels))
    val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
    val_f1 = f1_score(val_labels, val_preds, average='macro', zero_division=0)

    print(f"\nEpoch {epoch+1} Summary")
    print(f"Train | Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | Prec: {train_precision:.4f} | Rec: {train_recall:.4f} | F1: {train_f1:.4f}")
    print(f"Val   | Acc: {val_acc:.4f} | Prec: {val_precision:.4f} | Rec: {val_recall:.4f} | F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model_rn.state_dict(), MODEL_PATH)
        counter = 0
        print("Model improved. Saved.")
    else:
        counter += 1
        print(f"No improvement. Patience: {counter}/{PATIENCE}")

        if counter >= PATIENCE:
            print("Early stopping triggered")

Epoch 1/15 | Batch 282/282 | Loss: 0.7377 | Acc: 0.6875 | Prec: 0.6250 | Rec: 0.7083 | F1: 0.6063

Epoch 1 Summary
Train | Loss: 0.8510 | Acc: 0.6707 | Prec: 0.6725 | Rec: 0.6707 | F1: 0.6709
Val   | Acc: 0.6862 | Prec: 0.6260 | Rec: 0.6961 | F1: 0.6454
Model improved. Saved.
Epoch 2/15 | Batch 282/282 | Loss: 0.6428 | Acc: 0.8125 | Prec: 0.8056 | Rec: 0.8333 | F1: 0.7889

Epoch 2 Summary
Train | Loss: 0.8459 | Acc: 0.6755 | Prec: 0.6770 | Rec: 0.6755 | F1: 0.6755
Val   | Acc: 0.6798 | Prec: 0.6176 | Rec: 0.6911 | F1: 0.6337
No improvement. Patience: 1/5
Epoch 3/15 | Batch 282/282 | Loss: 1.2845 | Acc: 0.5625 | Prec: 0.5694 | Rec: 0.5056 | F1: 0.4694

Epoch 3 Summary
Train | Loss: 0.8415 | Acc: 0.6757 | Prec: 0.6767 | Rec: 0.6757 | F1: 0.6755
Val   | Acc: 0.6885 | Prec: 0.6292 | Rec: 0.6992 | F1: 0.6450
No improvement. Patience: 2/5
Epoch 4/15 | Batch 282/282 | Loss: 0.8357 | Acc: 0.6875 | Prec: 0.5833 | Rec: 0.5417 | F1: 0.5595

Epoch 4 Summary
Train | Loss: 0.8333 | Acc: 0.6772 | Pre